<a href="https://colab.research.google.com/github/wooogong/Reinforcement-Learning/blob/main/ex_gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gym
import numpy as np

In [ ]:
env = gym.make('Blackjack-v0')
print(env.observation_space)
print(env.action_space)

In [ ]:
for i in range(10):
  state = env.reset()
  #print(state)
  while True:
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    print(state, action, reward, next_state)

    if done:
      print("done")
      break
      state = next_state

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython import display


In [ ]:
def random_policy(observation):
  return env.action_space.sample()

def hit_only_policy(observation):
    if observation[0] == 21:
      return 0
    return 1

def stick_only_policy(observation):
  return 0

def fixed_policy(observation):
  score, dealer_score, usable_ace = observation
  return 0 if score >= 16 else 1

#V_tmp = mc_prediction(fixed_policy, env, num_episodes = 1)



In [ ]:
def play_game(policy, env):
  nWon = 0
  nLost = 0
  nDraw = 0
  policy = policy

  for i in range(100000):
    observation = env.reset()
    for i in range(100):
      action = policy(observation)

      next_observation, reward, done, _ = env.step(action)

      if done:

        if reward == 1:
          nWon += 1
        elif reward == -1:
          nLost += 1
        else:
          nDraw += 1
        break
      observation = next_observation
    print(nWon, nLost, nDraw)

In [ ]:
play_game(random_policy, env)
play_game(hit_only_policy, env)
play_game(stick_only_policy, env)

In [ ]:
from collections import defaultdict

def mc_prediction(policy, env, num_episodes, alpha = 0.01, gamma = 0.99):
  N = defaultdict(float)
  V = defaultdict(float)

  for i_episode in range(num_episodes):
    state = env.reset()
    trajectory = []
    while True:
      action = policy(state)
      next_state, reward, done, _ = env.step(action)
      trajectory.append((state, action, reward))

      if done:
        break

      state = next_state

    states, actions, rewards = zip(*trajectory)
    discounts = np.array([gamma**i for i in range(len(rewards))])

    for i, s in enumerate(states):
      N[s] += 1
      G = sum(rewards[i:]*discounts[:len(rewards) - 1])

      V[s] = V[s] + 1/N[s] * (G-V[s])

    return V


In [ ]:
V_random = mc_prediction(random_policy, env, num_episodes = 100000)
#plot_value_function(V_random, tilte = '100000 Steps')